In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 3.5MB/s 
     |████████████████████████████████| 61kB 20.5MB/s 
     |████████████████████████████████| 471kB 40.4MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536269 sha256=8e21a2503276d61c76abc22e558ee6fce6a957abaa1e9f936fde01c0c9ef225f
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=c728a8d1ead6e6223a9f576b47edf16349e81d8e8e84917649fce4b7bc159900
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=27532018e05065d2025560c924f80fdd7bff57f2492007e7543c1acb0f34fd3c
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
__global__ void sum_atomic(int nx, int *sum, int *data){
  int x = threadIdx.x + blockDim.x * blockIdx.x;
  if (x < nx){
     atomicAdd(sum, data[x]);
  }
}

""")

In [ ]:
atomic_sum = module.get_function("sum_atomic")

In [ ]:
nx = np.int32(10)
arr = np.arange(nx, dtype=np.int32)

In [ ]:
sum_gpu = gpuarray.zeros(1, dtype=np.int32)
arr_gpu = gpuarray.to_gpu(arr)

In [ ]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(nx / threads_per_block[0]), 1, 1)

In [ ]:
atomic_sum(nx, sum_gpu, arr_gpu, block=threads_per_block, grid=blocks_per_grid)

In [ ]:
sum_gpu.get()

array([45], dtype=int32)

In [ ]:
np.sum(arr)

45